# Qualitative sensitivity analysis

We showcase the use of `econsa` for qualitative sensitivity analysis.

In [2]:
# from temfpy.uncertainty_quantification import simple_linear_function,
from morris import elementary_effects  # noqa: F401
import numpy as np
import pandas as pd

The module `morris` implements the extended Morris method as proposed by Ge & Menendez (2017). They extend the Morris method in the sense, that their algorithm takes dependency among inputs into account.

For illustration purposes consider the Morris method for independent inputs only.

Let $x = \{x_1, \dots, x_k\}$ denote a sample of values assigned to the $X_i$'s. $f(x)$ is then the model output obtained for the values in $x$. Now consider a second sample $x_{\Delta_i} = \{x_1, \dots, x_{i-1}, x_i + \Delta, x_{i+1}, \dots, x_k\}$ that is identical to $x$ up to input $x_i$ which is varied by $\Delta$. Then, one elementary effect for input $i$ is derived by

$$EE_i = \frac{f(x_{\Delta_i}) - f(x)}{\Delta}.$$


The above elementary effect is computed $N$ times, each for a varying $\Delta$. The actual sensitivity measures resulting from the Morris method are the mean, denoted by $\mu^\ast_i$, and the standard deviation, denoted by $\sigma_i$, taken from the $N$ elementary effects per input $i$.

$$\mu_i^\ast = \frac{1}{N} \sum_{r=1}^N \vert EE_{i, r} \vert$$

$$\sigma_i = \sqrt{\frac{1}{N-1} \sum_{r=1}^N (EE_{i, r} - \mu_i)^2}$$

The derivation of the extended Morris indices is more complicated and we get *four* sensitivity indices indstead of only two: *independent* and *full* Morris indices, $(\mu_i^{\ast,\ ind}, \mu_i^{\ast,\ full}, \sigma_i^{ind}, \sigma_i^{full})$, which are computed analogously to the Morris indices under input independence, but are based on different elementary effects:

- $EE_i^{ind}$ denotes *independent* elementary effects for input $i$, effects that exclude the contributions attributable to the dependence between input $X_i$ and $X_j$ for $i \neq j$, and

- $EE_i^{full}$ denotes *full* elementary effects for input $i$, that include the effects due to correlation with other inputs.

For applying the Morris method, we need to specify the following arguments:

- `func`: The model for which we want to calculate the Morris indices. Note how the data needs to be accessed within the function. See below example.

- `params`: The mean values of the inputs.

- `cov`: The variance-covariance matrix of the inputs.

- `n_draws`: Number of draws. The total computational cost of the extended Morris method amounts to $3kN$, where $k$ denotes the number of inputs and $N$ the number of draws (`n_draws`).

Note that the current implementation of the Morris method in `econsa` does allow for Gaussian (i.e. normally distributed) inputs only.

Specify the input names in the data frames `params` and `cov` to display the input names in the output of `elementary_effects`.

In [3]:
params = pd.DataFrame(data=np.array([5.345, 0.0135, 2.15]), columns=['value'], index=['m', 'c', 's'])
cov = pd.DataFrame(data=np.diag([1, 0.000001, 0.01]), index=['m', 'c', 's'])

The model implemented by `func` needs to access the inputs in the following way, if the input names are specified in `params` and `cov`:

`m = x["value"]['m']`

`c = x["value"]['c']`

`s = x["value"]['s']`.

Alternatively we can access them via the index as well:

`m = x["value"][0]`

`c = x["value"][1]`

`s = x["value"][2]`.

In [7]:
def eoq_model_morris(x, r=0.1):
    """EOQ Model that accesses data as expected by elementary_effects."""
    m = x["value"]['m']
    c = x["value"]['c']
    s = x["value"]['s']

    # Need to ensure that there exists a solution (i.e. no NaNs).
    if m < 0:
        m = 0
    elif c < 0:
        raise ValueError
    elif s < 0:
        s = 0
    else:
        pass

    return np.sqrt((24 * m * s) / (r * c))

In [8]:
n_draws = 100
results = elementary_effects(eoq_model_morris, params, cov, n_draws)

The output of `elementary_effects` is a dictionary containing the four sensitivity indices derived from the `n_draws` elementary effects: $(\mu_i^{\ast,\ ind}, \mu_i^{\ast,\ full}, \sigma_i^{ind}, \sigma_i^{full})$.

In [9]:
results

{'mu_ind': m    152.932694
 c     57.953365
 s     33.104776
 dtype: float64,
 'mu_corr': m    2369.122666
 c    3270.066313
 s    2888.326540
 dtype: float64,
 'sigma_ind': m    54.844400
 c    17.314969
 s     7.963128
 dtype: float64,
 'sigma_corr': m    2523.492389
 c    8210.375153
 s    3814.720109
 dtype: float64,
 'mu_ind_cum':               m          c          s
 1           NaN        NaN        NaN
 2    124.901013  51.672836  26.198816
 3    125.806619  48.973670  29.796229
 4    123.068657  49.028250  31.225234
 5    119.660578  52.598716  30.243689
 ..          ...        ...        ...
 96   153.797320  57.495222  32.879575
 97   153.343157  57.987376  32.921207
 98   153.497392  57.815867  32.967114
 99   153.240693  58.075774  32.950881
 100  152.932694  57.953365  33.104776
 
 [100 rows x 3 columns],
 'mu_corr_cum':                m            c            s
 1            NaN          NaN          NaN
 2    1975.552648  2563.129032  1133.631188
 3    1533.898849  64